In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

import src
from src.reload import deep_reload

In [2]:
'''
Loading processed data
'''
deep_reload(src)

assets_path = 'Data/US/Processed/combined_assets.json'
lines_path = 'Data/US/Processed/lines.json'
profiles_path = 'Data/US/Processed/profiles/'
policies_path = 'Data/US/Processed/policies.json'

assets = src.utilities.read_json(assets_path)
lines = src.utilities.read_json(lines_path)
profiles = src.utilities.read_jsons(profiles_path, output = 'dict')
policies = src.utilities.read_json(policies_path)

In [3]:
path = 'Outputs/RPS.json'

with open(path, 'w') as file:
    
    json.dump(policies, file, indent = 4)

In [4]:
'''
Building the US graph
'''
deep_reload(src)

graph = src.build.build_graph(assets, lines, profiles)

In [5]:
graph.number_of_nodes(), graph.number_of_edges()

(67, 4288)

In [6]:
path = 'Outputs/US.json'

src.graph.graph_to_json(graph, path)

In [7]:
graph.nodes

NodeView(('ERC_FRNT', 'ERC_GWAY', 'ERC_PHDL', 'ERC_REST', 'ERC_WEST', 'FRCC', 'MIS_AMSO', 'MIS_AR', 'MIS_D_MS', 'MIS_IA', 'MIS_IL', 'MIS_INKY', 'MIS_LA', 'MIS_LMI', 'MIS_MAPP', 'MIS_MIDA', 'MIS_MNWI', 'MIS_MO', 'MIS_WOTA', 'MIS_WUMS', 'NENGREST', 'NENG_CT', 'NENG_ME', 'NY_Z_A', 'NY_Z_B', 'NY_Z_C&E', 'NY_Z_D', 'NY_Z_F', 'NY_Z_G-I', 'NY_Z_J', 'NY_Z_K', 'PJM_AP', 'PJM_ATSI', 'PJM_COMD', 'PJM_Dom', 'PJM_EMAC', 'PJM_PENE', 'PJM_SMAC', 'PJM_WMAC', 'PJM_West', 'SPP_KIAM', 'SPP_N', 'SPP_NEBR', 'SPP_SPS', 'SPP_WAUE', 'SPP_WEST', 'S_C_KY', 'S_C_TVA', 'S_D_AECI', 'S_SOU', 'S_VACA', 'WECC_AZ', 'WECC_CO', 'WECC_ID', 'WECC_IID', 'WECC_MT', 'WECC_NM', 'WECC_NNV', 'WECC_PNW', 'WECC_SCE', 'WECC_SNV', 'WECC_UT', 'WECC_WY', 'WEC_BANC', 'WEC_CALN', 'WEC_LADW', 'WEC_SDGE'))

In [8]:
'''
Interconnect Region Subgraphs
'''
from copy import deepcopy

regions = ['ERC', 'FRCC', 'MIS_', 'NENG', 'NY', 'PJM', 'SPP', 'S_', 'WEC']

for region in regions:

    subgraph_nodes = [n for n in graph.nodes if region in n]
    
    subgraph = src.graph.subgraph(graph, subgraph_nodes)
    
    path = f"Outputs/{region.replace('_', '')}.json"
    
    src.graph.graph_to_json(subgraph, path)